# [Report Link Here](https://docs.google.com/spreadsheets/d/1n8Lw-xK0IWOTPOCbhf4zNRu5x7uG7eZQqTCgIxq7sZo/edit?usp=sharing)

# Create Previous Event Attendees Table

In [ ]:
/* Previous event attendees - to run before PTG */

DROP TABLE IF EXISTS states_ia_reporting.previous_event_attendee;
CREATE TABLE states_ia_reporting.previous_event_attendee AS

(select distinct(myc_van_id)
from my_state_van.coord20_myc_003_event_shifts
where (reporting_week_start_date != current_date and reporting_week_start_date < current_date - 8)
	and state_code = 'IA'
    and (event_type = 'Candidate/Principal'
         or event_type = 'Canvass'
         or event_type = 'Community Event'
         or event_type = 'GOTV'
         or event_type = 'House Party'
         or event_type = 'Meeting'
         or event_type = 'Phone Bank'
         or event_type = 'Text Bank'
         or event_type = 'Training'
         or event_type = 'Voter Registration'
         or event_type = 'Vol Recruitment')
    and (event_role_name = 'Attendee'
         or event_role_name = 'Canvasser'
         or event_role_name = 'Dialer'
         or event_role_name = 'Host'
         or event_role_name = 'Phonebanker'
         or event_role_name = 'Texter'
         or event_role_name = 'Trainee'
         or event_role_name = 'Volunteer')
    and current_shift_status_name = 'Completed')

# Create Previous VBM Commits Table

In [ ]:
/* Previous weeks VBM commits - to be run before PTG */

DROP TABLE IF EXISTS states_ia_reporting.previous_vbm_commits;
CREATE TABLE states_ia_reporting.previous_vbm_commits AS 

(select myv_van_id
from my_state_van.coord20_myv_001_responses
where state_code = 'IA' and 
	survey_question_id = 406163 /* IA VBM PLAN SQ */
    and (survey_response_id = 1660740 /* Y: Commit Send ABR */
         or survey_response_id = 1660742 /* Y: Commit to Request */
         or survey_response_id = 1660745 /* Y: Commit to Return */
         or survey_response_id = 1660747 /* Y: Already Returned	 */
         or survey_response_id = 1660748) /* Y: ABR Collected */
    and reporting_week_start_date != current_date and reporting_week_start_date < current_date - 8)

# Create Table of Staff MYC VAN ID

In [ ]:
DROP TABLE IF EXISTS states_ia_reporting.staff_ac_myc_van_id;

CREATE TABLE states_ia_reporting.staff_ac_myc_van_id AS

(select distinct(myc_van_id)
from my_state_van.coord20_myc_002_activist_codes
where activist_code_id = 4722128)

# Complete Phase 1 PTG Metric Export

In [ ]:
/* In-Universe VBM Commits */ 
(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    vbm_table.fo_name,
    vbm_table.team_name,
    vbm_table.region_name,
    vbm_table.reporting_week_start_date,
    'In-Universe VBM Commits' as metric,
    count(vbm_table.myv_van_id) as metric_count
from (select t1.myv_van_id,
    t1.fo_name,
    t1.team_name,
    t1.region_name,
    t1.reporting_week_start_date,
    t1.survey_question_id,
    t1.survey_response_id
from my_state_van.coord20_myv_001_responses t1
join states_ia_universes.current_universe_reporting t2 on t1.myv_van_id = t2.myv_van_id --in universe commits
where state_code = 'IA' 
 and sq_most_recent_response = 1
 and survey_question_id = 406163 -- IA VBM PLAN SQ
 and (survey_response_id = 1660740 -- Y: Commit Send ABR 
      or survey_response_id = 1660742 -- Y: Commit to Request 
      or survey_response_id = 1660745)) vbm_table -- Y: Commit to Return 
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Virtual 1:1 Meetings */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    fo_name,
    team_name,
    region_name,
    reporting_week_start_date,
    'Virtual 1:1 Meetings' as metric,
    count(event_id) as metric_count
from my_state_van.coord20_myc_003_event_shifts
where event_type = 'One on One' 
    and (event_role_name = '1:1 Escalation'
        or event_role_name = '1:1 Introduction'
        or event_role_name = '1:1 Maintenance')
    and current_shift_status_name = 'Completed'
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Vote Leads */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    vl_table.fo_name,
    vl_table.team_name,
    vl_table.region_name,
    vl_table.reporting_week_start_date,
    'Vote Leads' as metric,
    count(distinct vl_table.myc_van_id) as metric_count
from
(select myc_van_id,
    fo_name,
 	region_name,
 	team_name,
    reporting_week_start_date,
    survey_question_id,
    survey_response_id
from my_state_van.coord20_myc_002_responses
where state_code = 'IA' and survey_question_id = 404815 and (survey_response_id = 1655271 or survey_response_id = 1655275)
and sq_most_recent_response=1) vl_table
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Unique Event Attendees */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    t1.fo_name,
    t1.team_name,
    t1.region_name,
    t1.reporting_week_start_date,
    'New Unique Event Attendees' as metric,
    count(distinct t1.myc_van_id) as metric_count
from my_state_van.coord20_myc_003_event_shifts t1
left join 
(select distinct(myc_van_id)
from my_state_van.coord20_myc_003_event_shifts
where (reporting_week_start_date < '2020-07-31')
	and state_code = 'IA'
    and (event_type = 'Candidate/Principal'
         or event_type = 'Canvass'
         or event_type = 'Community Event'
         or event_type = 'GOTV'
         or event_type = 'House Party'
         or event_type = 'Meeting'
         or event_type = 'Phone Bank'
         or event_type = 'Text Bank'
         or event_type = 'Training'
         or event_type = 'Voter Registration'
         or event_type = 'Vol Recruitment')
    and (event_role_name = 'Attendee'
         or event_role_name = 'Canvasser'
         or event_role_name = 'Dialer'
         or event_role_name = 'Host'
         or event_role_name = 'Phonebanker'
         or event_role_name = 'Texter'
         or event_role_name = 'Trainee'
         or event_role_name = 'Volunteer')
    and current_shift_status_name = 'Completed') t2 on t1.myc_van_id = t2.myc_van_id
left join states_ia_reporting.staff_ac_myc_van_id t3 on t1.myc_van_id = t3.myc_van_id
left join states_ia_reporting.events_to_suppress t4 on t1.event_id = t4.event_id
where t2.myc_van_id is null
 	and t3.myc_van_id is null
 	and t4.event_id is null
    and (t1.event_type = 'Candidate/Principal'
         or t1.event_type = 'Canvass'
         or t1.event_type = 'Community Event'
         or t1.event_type = 'GOTV'
         or t1.event_type = 'House Party'
         or t1.event_type = 'Meeting'
         or t1.event_type = 'Phone Bank'
         or t1.event_type = 'Text Bank'
         or t1.event_type = 'Training'
         or t1.event_type = 'Voter Registration'
         or t1.event_type = 'Vol Recruitment')
    and (t1.event_role_name = 'Attendee'
        or t1.event_role_name = 'Canvasser'
        or t1.event_role_name = 'Dialer'
        or t1.event_role_name = 'Host'
        or t1.event_role_name = 'Phonebanker'
        or t1.event_role_name = 'Texter'
        or t1.event_role_name = 'Trainee'
        or t1.event_role_name = 'Volunteer')
    and t1.current_shift_status_name = 'Completed'
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Recruitment Calls */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	fo_name,
	team_name,
    region_name,
    reporting_week_start_date,
    'Recruitment Calls' as metric,
    count(myc_van_id) as metric_count
from my_state_van.coord20_myc_002_contacts
where reporting_week_start_date >= '2020-07-31'
	and contact_type_name = 'Phone'
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

# Weekly Phase 1 PTG Metric Export

In [ ]:
/* VBM Commits */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    vbm_table.fo_name,
    vbm_table.team_name,
    vbm_table.region_name,
    vbm_table.reporting_week_start_date,
    'In-Universe VBM Commits' as metric,
    count(vbm_table.myv_van_id) as metric_count
from (select t3.myv_van_id,
    t1.fo_name,
    t1.team_name,
    t1.region_name,
    t1.reporting_week_start_date,
    t1.survey_question_id,
    t1.survey_response_id
from my_state_van.coord20_myv_001_responses t1
join states_ia_universes.current_universe_reporting t3 on t1.myv_van_id = t3.myv_van_id --in universe commits
left join states_ia_reporting.previous_vbm_commits t2 on t1.myv_van_id = t2.myv_van_id
--left join states_ia_reporting.bad_vbm_myv_ids_08102020 t3 on t1.myv_van_id = t3.voter_file_vanid
where state_code = 'IA' 
 and t2.myv_van_id is null /* Eliminates previous VBM commits */
-- and t3.voter_file_vanid is null /* Eliminates bad VBM commits */
 and survey_question_id = 406163 -- IA VBM PLAN SQ
 and (survey_response_id = 1660740 -- Y: Commit Send ABR 
      or survey_response_id = 1660742 -- Y: Commit to Request 
      or survey_response_id = 1660745)) vbm_table -- Y: Commit to Return 
 
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Virtual 1:1 Meetings */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    fo_name,
    team_name,
    region_name,
    reporting_week_start_date,
    'Virtual 1:1 Meetings' as metric,
    count(event_id) as metric_count
from my_state_van.coord20_myc_003_event_shifts
where current_date < reporting_week_start_date + 8
	and current_date > reporting_week_start_date 
    and event_type = 'One on One' 
    and (event_role_name = '1:1 Escalation'
        or event_role_name = '1:1 Introduction'
        or event_role_name = '1:1 Maintenance')
    and current_shift_status_name = 'Completed'
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Unique Event Attendees */
(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    t1.fo_name,
    t1.team_name,
    t1.region_name,
    t1.reporting_week_start_date,
    'New Unique Event Attendees' as metric,
    count(distinct t1.myc_van_id) as metric_count
from my_state_van.coord20_myc_003_event_shifts t1
left join states_ia_reporting.previous_event_attendee t2 on t1.myc_van_id = t2.myc_van_id
left join states_ia_reporting.staff_ac_myc_van_id t3 on t1.myc_van_id = t3.myc_van_id
left join states_ia_reporting.events_to_suppress t4 on t1.event_id = t4.event_id
where t2.myc_van_id is null
 	and t3.myc_van_id is null
 	and t4.event_id is null
	and current_date < t1.reporting_week_start_date + 8 
	and current_date > t1.reporting_week_start_date
    and (t1.event_type = 'Candidate/Principal'
         or t1.event_type = 'Canvass'
         or t1.event_type = 'Community Event'
         or t1.event_type = 'GOTV'
         or t1.event_type = 'House Party'
         or t1.event_type = 'Meeting'
         or t1.event_type = 'Phone Bank'
         or t1.event_type = 'Text Bank'
         or t1.event_type = 'Training'
         or t1.event_type = 'Voter Registration'
         or t1.event_type = 'Vol Recruitment')
    and (t1.event_role_name = 'Attendee'
        or t1.event_role_name = 'Canvasser'
        or t1.event_role_name = 'Dialer'
        or t1.event_role_name = 'Host'
        or t1.event_role_name = 'Phonebanker'
        or t1.event_role_name = 'Texter'
        or t1.event_role_name = 'Trainee'
        or t1.event_role_name = 'Volunteer')
    and t1.current_shift_status_name = 'Completed'
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Vote Leads */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    vl_table.fo_name,
    vl_table.team_name,
    vl_table.region_name,
    vl_table.reporting_week_start_date,
    'Vote Leads' as metric,
    count(distinct vl_table.myc_van_id) as metric_count
from
(select myc_van_id,
    fo_name,
 	region_name,
 	team_name,
    reporting_week_start_date,
    survey_question_id,
    survey_response_id
from my_state_van.coord20_myc_002_responses
where state_code = 'IA' and survey_question_id = 404815 and (survey_response_id = 1655271 or survey_response_id = 1655275 or survey_response_id = 1697864)
and sq_most_recent_response=1) vl_table
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)

union all

/* Recruitment Calls */

(select 'PTG Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	fo_name,
	team_name,
    region_name,
    reporting_week_start_date,
    'Recruitment Calls' as metric,
    count(myc_van_id) as metric_count
from my_state_van.coord20_myc_002_contacts
where (reporting_week_start_date > current_date - 8 and reporting_week_start_date < current_date)
	and contact_type_name = 'Phone'
group by 1, 2, 3, 4, 5, 6
order by 2, 3, 4, 5)